In [1]:
%load_ext autoreload
%autoreload 2
import os.path

path = ''
if not os.path.isfile('001_Data_retrieve.ipynb'):
    !pip install -U -q PyDrive
    !pip install  transformers 
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/TP-FINAL/bioinformatics_final_project'
    !pip install rdkit-pypi

    !wget https://raw.githubusercontent.com/GuilleBur/tp_final_bioinformatica/main/dataaug.py
    !wget https://raw.githubusercontent.com/GuilleBur/tp_final_bioinformatica/main/datagen.py
    !wget https://raw.githubusercontent.com/GuilleBur/tp_final_bioinformatica/main/smiles_tokenizer.py

# Modelo CNN con generador, embedings de smiles y data-augmentation

In [3]:
!pip install rdkit-pypi

ERROR: Could not find a version that satisfies the requirement rdkit-pypi
ERROR: No matching distribution found for rdkit-pypi


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datagen import smiles_dict, smiles_to_seq
from dataaug import SmilesEnumerator

### smiles_dict

In [ ]:
len(smiles_dict)

42

smiles_dict nos da un tokenizador para simplificar el problema. Puede ver como se construyó en la notebook **deep_chem**.
Si al momento de correr el modelo con este diccionario encuentra problemas de key_error, puede agregar los faltantes al diccionario

Mirar dentro de **datagen.py** como se usa este diccionario con la función **smiles_to_seq** para tokenizar. El código es muy sencillo

In [ ]:
print(smiles_dict)

{'#': 1, '(': 2, ')': 3, '+': 43, '-': 5, '/': 6, '1': 7, '2': 8, '3': 9, '4': 10, '5': 11, '6': 12, '7': 13, '8': 14, '=': 15, 'C': 16, 'F': 17, 'H': 18, 'I': 19, 'N': 20, 'O': 21, 'P': 22, 'S': 23, '[': 24, '\\': 25, ']': 26, '_': 27, 'c': 28, 'Cl': 29, 'Br': 30, 'n': 31, 'o': 32, 's': 33, '@': 34, '.': 35, 'a': 36, 'B': 37, 'e': 38, 'i': 39, '9': 40, '10': 41, '11': 42}


# Carga de los datos

In [ ]:
df = pd.read_csv(path+'/data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv')
#df = pd.read_csv('/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/TP-FINAL/bioinformatics_final_project/data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

In [ ]:
max_len_idx = df['canonical_smiles'].apply(len).argmax()
min_len_idx = df['canonical_smiles'].apply(len).argmin()
max_sequence_len = len(df['canonical_smiles'].iloc[max_len_idx]) + 20

In [ ]:
df.head()

,molecule_chembl_id,canonical_smiles,standard_value,standard_value_norm,pIC50
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0,750.0,6.124939
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0,100.0,7.000000
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.0,50000.0,4.301030
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.0,300.0,6.522879
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.0,800.0,6.096910


In [ ]:
X = df['canonical_smiles'].values
y = df['pIC50'].values

# Data augmentation:

https://arxiv.org/pdf/1703.07076.pdf

https://github.com/EBjerrum/molvecgen

https://github.com/Ebjerrum/SMILES-enumeration

En la publicación de arriba se describe una técnica de aumentación de datos para los smiles. Leerla si es de su interes (Opcional)

En el módulo **dataug.py**, tomando como referencia los repositorios arriba citados se implementó la aumentación de datos

In [ ]:
from dataaug import SmilesEnumerator
sme = SmilesEnumerator()
for i in range(10):
    print(sme.randomize_smiles('CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1'))
    

c1c(OCc2ccccc2)cc(-n2nc(OCC)oc2=O)cc1
o1c(=O)n(-c2cc(OCc3ccccc3)ccc2)nc1OCC
O(Cc1ccccc1)c1cc(-n2c(=O)oc(OCC)n2)ccc1
c1(-n2nc(OCC)oc2=O)cccc(OCc2ccccc2)c1
o1c(=O)n(-c2cccc(OCc3ccccc3)c2)nc1OCC
C(c1ccccc1)Oc1cccc(-n2c(=O)oc(OCC)n2)c1
c1ccccc1COc1cc(-n2c(=O)oc(OCC)n2)ccc1
C(C)Oc1oc(=O)n(-c2cccc(OCc3ccccc3)c2)n1
c1cc(COc2cc(-n3c(=O)oc(OCC)n3)ccc2)ccc1
c1cc(-n2nc(OCC)oc2=O)cc(OCc2ccccc2)c1


# DataGenerator

Construir un generador al que se le pase al instanciarlo:
- X: smiles (formula química)
- y: pIC50
- batch_size
- max_sequence_len (int): La máxima longitud de las secuencias (para hacer el padding)
- data_augmentation (boolean): si quiero hacer o no data-augmentation. 
- shuffle (boolean)

Guardarlo en el módulo **datagen.py** con el nombre de la clase **DataGenerator**

Notar que el módulo **datagen.py** ya tiene una estructura para completar

### Importamos el módulo y lo probamos

In [ ]:
from datagen import DataGenerator

In [ ]:
dgen = DataGenerator(X, y, max_sequence_len, batch_size=16, data_augmentation=False)

In [ ]:
len(dgen) * dgen.batch_size

5104

# Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(X_train), len(y_train), len(X_test)

(4082, 4082, 1021)

In [ ]:
dgen_train = DataGenerator(X_train, y_train, seq_length=max_sequence_len, batch_size=128, data_augmentation=True)
dgen_test = DataGenerator(X_test, y_test, seq_length=max_sequence_len, batch_size=128, data_augmentation=False)

In [ ]:
for i, (X_b, y_b) in enumerate(dgen_train):
    print(f'{i}\r', end='')

# Network Model

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout, Activation, BatchNormalization, Conv1D, MaxPool1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Flatten, Input, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras import layers as L

In [ ]:
# Implementar modelo de TextCNN
def text_cnn_1d(sequence_length, vocab_size, embedding_size, filter_sizes, num_filters):
    # Recordar que estamos en la funcional model API y hay que entender que conectar con que.
    input_x = Input(shape=(sequence_length,), name='input_x')
    embedding_layer = Embedding(vocab_size+1, embedding_size, input_length=max_sequence_len)(input_x)
    pooled_outputs = []
    for filter_size in filter_sizes:
        conv1D = Conv1D( num_filters, filter_size, activation='relu', padding='same')(embedding_layer)
        max_p = MaxPool1D()(conv1D)
        pooled_outputs.append(max_p)
        
    h_pool = Concatenate(axis=2)(pooled_outputs)
    h_pool = Flatten()(h_pool)  #Flatten porque tengo MaxPool1D en vez de GlobalMaxPooling1D?
    dense = Dense(100, activation='relu')(h_pool)
    dense = Dense(50, activation='relu')(dense)
    dense = Dense(1)(dense) # Salida
    model = tf.keras.models.Model(input_x, dense)
    return model

In [ ]:
# Puede cambiar estos hiperparámtros. Son solo de referencia
FILTER_SIZES = (3, 4, 5)
NUM_FILTERS = 128
vocab_size = len(smiles_dict)+1
embeddings_size = 128 # 2

In [ ]:
model = text_cnn_1d(max_sequence_len, vocab_size, embeddings_size, FILTER_SIZES, NUM_FILTERS)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_x (InputLayer)            [(None, 227)]        0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 227, 128)     5632        input_x[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 227, 128)     49280       embedding_5[0][0]                
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 227, 128)     65664       embedding_5[0][0]                
____________________________________________________________________________________________

In [ ]:
def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=[R2])

In [ ]:
#Callbacks
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2)
mcp = ModelCheckpoint(path+'/models/best_model', save_best_only=True, save_format="h5")

### PRUEBAS REALIZADAS
- 1) Con data Agumentation
---
> `val_R2= 0.56 `<br>
> FILTER_SIZES = (3, 4, 5) <br>
> NUM_FILTERS = 128  <br>
> embeddings_size = 128 # 2 <br>
---


- 2) Sin data Agumentation
---
> `val_R2= 0.41 `<br>
> FILTER_SIZES = (3, 4, 5) <br>
> NUM_FILTERS = 128  <br>
> embeddings_size = 128 # 2 <br>
---
- 3) Modificando la Topologia  (aplicando GlobalMaxPooling) 
---
> `val_R2= 0.41 `<br>
> FILTER_SIZES = (3, 4, 5) <br>
> NUM_FILTERS = 128  <br>
> embeddings_size = 128 # 2 <br>
---
- 4) Usando MILES enumeration, vectorization and batch generation https://github.com/Ebjerrum/SMILES-enumeration  Sin embeddings, utilizando LSTM, CNN, ETC. Con Embeddings--> MLP + Embeddings + Attention,  MLP + Embeddings + Attention + CNN, Bidir RNN + Attention

In [ ]:
history = model.fit(dgen_train, epochs=2000, validation_data=dgen_test, callbacks=[earlystop, mcp])

Epoch 1/2000
32/32 [==============================] - 31s 933ms/step - loss: 11.9781 - R2: -3.2997 - val_loss: 2.8569 - val_R2: 0.0103
INFO:tensorflow:Assets written to: /content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/TP-FINAL/bioinformatics_final_project/models/best_model_1/assets
Epoch 2/2000
32/32 [==============================] - 32s 984ms/step - loss: 2.8586 - R2: -0.0261 - val_loss: 2.7689 - val_R2: 0.0407
INFO:tensorflow:Assets written to: /content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/TP-FINAL/bioinformatics_final_project/models/best_model_2/assets
Epoch 3/2000
32/32 [==============================] - 30s 936ms/step - loss: 2.6205 - R2: 0.0593 - val_loss: 2.6841 - val_R2: 0.0701
INFO:tensorflow:Assets written to: /content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/TP-FINAL/bioinformatics_final_project/models/best_model_3/assets
Epoch 4/2000
32/32 [==============================] - 32s 990ms/step - loss: 2.538

In [ ]:
#paso todo el dataGenerator de Test a un array
X_test_eval = []
y_t_eval = []
for X_t, y_t in dgen_test:
    X_test_eval = X_test_eval + [list(t) for t in X_t]
    y_t_eval = y_t_eval + list(y_t)
X_test_eval = np.array(X_test_eval)
y_test = np.array(y_t_eval)

In [ ]:
X_train.shape

(4082,)

In [ ]:
X_test_eval.shape

(8168, 227)

In [ ]:
X_test_eval.shape, y_test.shape

((8168, 227), (8168,))

In [ ]:
y_pred = model.predict(X_test_eval)

# Score

In [ ]:
r2 = 1 - ((y_test - y_pred.reshape(-1)) ** 2).sum() / ((y_test - y_test.mean()) ** 2).sum()

In [ ]:
r2


0.5419384260925506

In [ ]:
filename = 'models/embed_128_text_cnn_aug_100x100_0.0001.h5'
model.save(filename)

In [ ]:
# model.summary()

# Load model (solo si se grabo el modelo)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model_loaded = load_model(filename, custom_objects={'R2': R2})

In [ ]:
y_pred = model_loaded.predict(X_test_eval)

In [ ]:
r2 = 1 - ((y_test - y_pred.reshape(-1)) ** 2).sum() / ((y_test - y_test.mean()) ** 2).sum()
r2

# Evaluación

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(color_codes=True)
sns.set_style("white")

ax = sns.regplot(x=y_test, y=y_pred, scatter_kws={'alpha':0.4})
ax.set_xlabel('Experimental pIC50', fontsize='large', fontweight='bold')
ax.set_ylabel('Predicted pIC50', fontsize='large', fontweight='bold')
ax.set_xlim(0, 12)
ax.set_ylim(0, 12)
ax.figure.set_size_inches(5, 5)
plt.show

In [ ]:
smiles_dict = {'#': 1, '(': 2, ')': 3, '+': 4, '-': 5, '/': 6, '1': 7, '2': 8, '3': 9, '4': 10, '5': 11, '6': 12, '7': 13, '8': 14, '=': 15, 'C': 16, 'F': 17, 'H': 18, 'I': 19, 'N': 20, 'O': 21, 'P': 22, 'S': 23, '[': 24, '\\': 25, ']': 26, '_': 27, 'c': 28, 'Cl': 29, 'Br': 30, 'n': 31, 'o': 32, 's': 33, '@': 34, '.': 35, 'a': 36, 'B': 37, 'e': 38, 'i': 39, '9': 40, '10': 41, '11': 42, '+':43}


# Pasa 
def smiles_to_seq(smiles, seq_length, char_dict=smiles_dict):
    """ Tokenize characters in smiles to integers
    """
    smiles_len = len(smiles)
    seq = []
    keys = char_dict.keys()
    i = 0
    while i < smiles_len:
        # Skip all spaces
        if smiles[i:i + 1] == ' ':
            i = i + 1
        # For 'Cl', 'Br', etc.
        elif smiles[i:i + 2] in keys:
            seq.append(char_dict[smiles[i:i + 2]])
            i = i + 2
        elif smiles[i:i + 1] in keys:
            seq.append(char_dict[smiles[i:i + 1]])
            i = i + 1
        else:
            print(smiles)
            print(smiles[i:i + 1], i)
            raise ValueError('character not found in dict')
    for i in range(seq_length - len(seq)):
      # Padding with '_'
      seq.append(0)
    return seq

In [ ]:
sme.randomize_smiles(X[0])

In [ ]:
X[0]

In [ ]:
a = [smiles_to_seq(x, 207)   for x in X ] 

In [ ]:
len(a[0])

In [ ]:
indexes = np.arange(len(X))
indexes.shape

In [ ]:
indexes = indexes[0*10:(0+1)*10]

In [ ]:
indexes

In [ ]:
X[indexes]

In [ ]:
a = [sme.randomize_smiles(smile) for smile in X]
npa = np.array(a)

In [ ]:
type(a)

In [ ]:
type(npa)

In [ ]:
npa.shape

In [ ]:
X[0]

In [ ]:
npa[0]

In [ ]:
np.random.randint(0,1)

In [ ]:
np.random.randint(2)

In [ ]:
if(np.random.randint(2)==0 ):
    print('OK')